# Evaluation Notebook

## Setup

In [ ]:
!pip install -r requirements.txt

## Download Data

either provide a download link here: https://drive.google.com/drive/folders/1SS4wRKzELXB4qXrmHg8T_wfbjYX1xdIf?usp=share_link  
sorry but gdown seemingly can't handle more than 50 files at a time and we have 40000

You need to keep the same structure as in the drive file to avoid any problem

## Your Plots and Results

In [4]:
from dataset import DatasetAmazon
from model import PlanetModel, testModel, ResNet
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
import time

from pytorch_lightning.callbacks import ModelCheckpoint
import os
import sys
import numpy as np

from accuracy_metrics import Hamming_distance, transform_pred, overall_acc, count_false

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

If you just want to see if it runs set:  
 - full, tiny = False, True  

Note: you will **NOT** retrieve our results with this option

In [ ]:
full, tiny = True, False
test_dataset = DatasetAmazon(full=full, tiny=tiny, test=True, path_to_labels="CSV/train_label_vector.pkl")
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size = 1, num_workers=4, shuffle=False) # change workers if you have more

### Pretrained model

In [ ]:
model_name = "ResNet18-epoch=48_over_50-val_accuracy=0.95.ckpt"
checkpoint_path = "../IPEO_Planet_project/checkpoints/"+model_name

pretrained = True
depth = 18
test_model = ResNet(depth=depth)
model = PlanetModel(model=test_model)
model.load_state_dict(torch.load(checkpoint_path)["state_dict"])

In [ ]:
csv_logger = pl_loggers.CSVLogger(save_dir="", name = "logs")

max_epochs = 50

checkpoint_callback = ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename=f'ResNet{depth}'+'-{epoch}_'+f'over_{max_epochs}'+'-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
)

In [ ]:
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="gpu", devices=[0], 
                     logger=csv_logger, callbacks=[checkpoint_callback],
                     resume_from_checkpoint=None, check_val_every_n_epoch=1)

In [ ]:
rs = trainer.predict(model, dataloaders=test_dl) # this can take a little bit of time

In [ ]:
false_neg, false_pos = np.zeros((1, 17)), np.zeros((1, 17))
y_hat, y = list(map(list, zip(*rs)))
overall_accuracy = 0.0
hamming_dist = 0.0
for i in range(len(y_hat)):
    overall_accuracy += overall_acc(y_hat[i], y[i])
    hamming_dist += Hamming_distance(y_hat[i], y[i])
    false_positive, false_negative = count_false(y_hat[i], y[i])
    false_pos += false_positive
    false_neg += false_negative
overall_accuracy = overall_accuracy/len(y_hat)
hamming_dist = hamming_dist/len(y_hat)
                                        
print(f"Overall accuracy: {overall_accuracy:.4f}\nHamming distance: {hamming_dist}")
print(false_pos, false_neg)

### Model from scratch

In [ ]:
model_name = "ResNet18-epoch=48_over_50-val_accuracy=0.95.ckpt"
checkpoint_path = "../IPEO_Planet_project/checkpoints/"+model_name

test_model = testModel(max_channels=512) 
model = PlanetModel(model=test_model)
model.load_state_dict(torch.load(checkpoint_path)["state_dict"])

In [ ]:
csv_logger = pl_loggers.CSVLogger(save_dir="", name = "logs")

max_epochs = 50


ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename='test-{epoch}_'+ f'over_{max_epochs}'+ '-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
    )

In [ ]:
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="gpu", devices=[0], 
                     logger=csv_logger, callbacks=[checkpoint_callback],
                     resume_from_checkpoint=None, check_val_every_n_epoch=1)

In [ ]:
rs = trainer.predict(model, dataloaders=test_dl) # this can take a little bit of time

In [ ]:
false_neg, false_pos = np.zeros((1, 17)), np.zeros((1, 17))
y_hat, y = list(map(list, zip(*rs)))
overall_accuracy = 0.0
hamming_dist = 0.0
for i in range(len(y_hat)):
    overall_accuracy += overall_acc(y_hat[i], y[i])
    hamming_dist += Hamming_distance(y_hat[i], y[i])
    false_positive, false_negative = count_false(y_hat[i], y[i])
    false_pos += false_positive
    false_neg += false_negative
overall_accuracy = overall_accuracy/len(y_hat)
hamming_dist = hamming_dist/len(y_hat)
                                        
print(f"Overall accuracy: {overall_accuracy:.4f}\nHamming distance: {hamming_dist}")
print(false_pos, false_neg)